# Introduction to Cirq

This exercise will be a gentle introduction to Cirq, software for writing, manipulating, and optimizing quantum circuits and then running them against quantum computers and simulators. More information on Cirq can be found on: https://ml2quantum.com/cirq/. We experiment with combining different quantum gates after eachother, and printing out the resulting circuits


In [4]:
import cirq

In [5]:
q0 = cirq.GridQubit(0,0)

qubits = [cirq.GridQubit(x,y) for x in range(3) for y in range(3)]
print(qubits)

[cirq.GridQubit(0, 0), cirq.GridQubit(0, 1), cirq.GridQubit(0, 2), cirq.GridQubit(1, 0), cirq.GridQubit(1, 1), cirq.GridQubit(1, 2), cirq.GridQubit(2, 0), cirq.GridQubit(2, 1), cirq.GridQubit(2, 2)]


In [10]:
circuit = cirq.Circuit()
circuit.append([cirq.H(q) for q in qubits if (q.row + q.col) % 2 == 0],
              strategy=cirq.InsertStrategy.EARLIEST)

circuit.append([cirq.X(q) for q in qubits if (q.row + q.col) % 2 == 1],
              strategy=cirq.InsertStrategy.NEW_THEN_INLINE)
print(circuit)

#gates apprearing on the same vertical line constitue a moment
#many different strategies, EARLIEST is the default
#below, the circuit has two moments

(0, 0): ───H───────

(0, 1): ───────X───

(0, 2): ───H───────

(1, 0): ───────X───

(1, 1): ───H───────

(1, 2): ───────X───

(2, 0): ───H───────

(2, 1): ───────X───

(2, 2): ───H───────


In [13]:
for i,m in enumerate(circuit):
    print("Moment {}: {}".format(i,m))
    
#so we can iterate over a Circuit's moments

Moment 0: H((0, 0)) and H((0, 2)) and H((1, 1)) and H((2, 0)) and H((2, 2))
Moment 1: X((0, 1)) and X((1, 0)) and X((1, 2)) and X((2, 1))


In [24]:
qubits = [cirq.GridQubit(x,y) for x in range(3) for y in range(3)]

print(qubits[0])

(0, 0)


In [25]:
x_gate = cirq.X
x_op = x_gate(qubits[0])

print(x_op)

X((0, 0))


In [28]:
cz = cirq.CZ(qubits[0],qubits[1])
x = cirq.X(qubits[2])
moment = cirq.Moment([x,cz])

print(moment)

X((0, 2)) and CZ((0, 0), (0, 1))


In [31]:
cz01 = cirq.CZ(qubits[0], qubits[1])
x2 = cirq.X(qubits[2])
cz12 = cirq.CZ(qubits[1], qubits[2])
moment0 = cirq.Moment([cz01, x2])
moment1 = cirq.Moment([cz12])
circuit = cirq.Circuit((moment0,moment1))

print(circuit)

(0, 0): ───@───────
           │
(0, 1): ───@───@───
               │
(0, 2): ───X───@───


In [43]:
from cirq.ops import CZ, H

q0, q1, q2 = [cirq.GridQubit(i,0) for i in range(3)]
circuit = cirq.Circuit()
circuit.append([CZ(q0,q1), H(q2)])

print(circuit)

ImportError: cannot import name 'C'

In [36]:
circuit.append([H(q0), CZ(q1,q2)])

print(circuit)

#if you run multiple times appends multiple times

(0, 0): ───@───H───H───H───H───
           │
(1, 0): ───@───@───@───@───@───
               │   │   │   │
(2, 0): ───H───@───@───@───@───


- decoherence
- connectivity
- vidality

Quantum algorithm implementations for Beginners arxiv 

# Exercise 1, Implement an OR gate over 2 Qubits

In [37]:
#we realize that OR = NOT (AND (NOT(X), NOT(Y)))
#moreover, we realize that AND can be represented by the Toffoli gate

In [41]:
cq1 = cirq.GridQubit(0,0)
cq2 = cirq.GridQubit(1,0)
tq = cirq.GridQubit(2,0)

Xcq1 = cirq.X(cq1)
Xcq2 = cirq.X(cq2)
tofandtq = cirq.TOFFOLI(cq1,cq2,tq)
Xtq = cirq.X(tq)

moment0 = cirq.Moment([Xcq1, Xcq2])
moment1 = cirq.Moment([tofandtq])
moment2 = cirq.Moment([Xtq])
circuit = cirq.Circuit((moment0,moment1,moment2))

print(circuit)


(0, 0): ───X───@───────
               │
(1, 0): ───X───@───────
               │
(2, 0): ───────X───X───


# Exercise 2, Represent an OR gate on 3 qubits

In [46]:
cq1 = cirq.GridQubit(0,0)
cq2 = cirq.GridQubit(1,0)
auxiliary_qubit = cirq.GridQubit(2,0)
cq3 = cirq.GridQubit(3,0)
tq = cirq.GridQubit(4,0)

Xcq1 = cirq.X(cq1)
Xcq2 = cirq.X(cq2)
Xcq3 = cirq.X(cq3)
tofandtq1 = cirq.TOFFOLI(cq1,cq2,auxiliary_qubit)
tofandtq2 = cirq.TOFFOLI(cq2, auxiliary_qubit, tq)
Xtq = cirq.X(tq)

moment0 = cirq.Moment([Xcq1, Xcq2, Xcq3])
moment1 = cirq.Moment([tofandtq1])
moment2 = cirq.Moment([tofandtq2])
moment3 = cirq.Moment([Xtq])
circuit = cirq.Circuit((moment0,moment1,moment2,moment3))

print(circuit)

(0, 0): ───X───@───────────
               │
(1, 0): ───X───@───@───────
               │   │
(2, 0): ───────X───@───────
                   │
(3, 0): ───X───────┼───────
                   │
(4, 0): ───────────X───X───
